# FULCRUM - TOURNAMENT

## Vali - keywords for Round 2

In [3]:
import parquet
import pandas as pd
import openpyxl

In [4]:
import json

In [6]:
import pandas as pd
pd_naics_keyw = pd.read_csv('naics_summary_keywords.csv')
pd_naics_keyw.head()

,Unnamed: 0,naics_code,naics_label,description,summary,keywords
0,0,111,Crop Production,Industries in the Crop Production subsector gr...,The Crop Production subsector comprises establ...,"Crop Production, farms, orchards, greenhouses,..."
1,1,112,Animal Production and Aquaculture,Industries in the Animal Production and Aquacu...,The Animal Production and Aquaculture subsecto...,"Animal Production, Aquaculture, Ranches, Farms..."
2,2,113,Forestry and Logging,Industries in the Forestry and Logging subsect...,The forestry and logging industries harvest ti...,"Forestry, Logging, Timber Production, Reforest..."
3,3,114,"Fishing, Hunting and Trapping","Industries in the Fishing, Hunting and Trappin...","The Fishing, Hunting and Trapping industry rel...","Fishing, Hunting, Trapping, Harvest, Wild Anim..."
4,4,115,Support Activities for Agriculture and Forestry,Industries in the Support Activities for Agric...,The Support Activities for Agriculture and For...,"Support Activities, Agriculture, Forestry, Sup..."


In [7]:
import pandas as pd
import spacy
import re
import string

In [21]:
nlp = spacy.load("en_core_web_lg")

def remove_mentions_and_hashtags(text):
    text = re.sub(r"@\w+", "", text)
    text = re.sub(r"#\w+", "", text)
    return text


def remove_numbers(text):
    text = re.sub(r"\d+", "", text)
    return text


def remove_punctuation(text):

    translator = str.maketrans("", "", string.punctuation)

    # Remove punctuation using the translation table
    text_without_punct = text.translate(translator)

    return text_without_punct


def remove_stopwords(text):
    filtered_sentence = []
    doc = nlp(text)
    for token in doc:
        if token.is_stop == False:
            filtered_sentence.append(token.text)
    return " ".join(filtered_sentence)


def clean_text(text,to_lemmatize:bool = True):

    # Standardize text
    # text = standardize_accented_chars(text)

    # Remove URLs
    text = re.sub(r"http\S+", "", text)

    # Remove mentions and hashtags
    text = remove_mentions_and_hashtags(text)

    # Lowercase
    text = text.lower()

    # Remove punctuation
    text = remove_punctuation(text)

    # Remove numbers
    text = remove_numbers(text)

    # Remove all the special characters
    text = re.sub(r"\W", " ", text)

    # Remove stopwords
    text = remove_stopwords(text)

    # Substituting multiple spaces with single space
    text = re.sub(r"\s+", " ", text, flags=re.I)

    if to_lemmatize:
        text = lemmatize(text)

    return text


def lemmatize(text):
    doc = nlp(text)
    text = " ".join([token.lemma_ for token in doc])
    return text


@lru_cache(maxsize=100)
def cached_similarity(keywords, tokens):
    return calculate_similarity(keywords, tokens)


# Calculate similarity
def calculate_similarity(keywords, tokens):
    
    text1 = nlp(" ".join(keywords))
    text2 = nlp(" ".join(tokens))
    
    return text1.similarity(text2)


import numpy as np


def predict_industry_label(keywords, industry_data):
    label_key = 'naics_label'
    code_key = 'naics_code'

    # Precompute similarities if possible
    similarities = industry_data["tokens"].apply(
        lambda tokens: cached_similarity(keywords, tokens)
    )

    # Find index of the maximum similarity
    max_index = similarities.idxmax()
    max_similarity = similarities[max_index]

    # predicted_label = industry_data.at[max_index, code_key]
    # predicted_label = str(predicted_label) + " " + str(industry_data.at[max_index, label_key])
    predicted_label = str(industry_data.at[max_index, label_key])

    return predicted_label, max_similarity

In [9]:

industry_data = pd_naics_keyw

# Preprocess industry descriptions
# industry_data['processed_description'] = industry_data['keywords'].str.lower() + industry_data['naics_label'].str.lower()
industry_data['processed_description'] = industry_data['keywords'].str.lower()
industry_data['processed_description'] = [clean_text(text) for text in industry_data['processed_description']]

# for i in range(len(industry_data['processed_description'])):
#     doc = nlp(industry_data['processed_description'][i])
#     chunks_text = [chunk.text for chunk in doc.noun_chunks][:5]
#     industry_data.loc[i, 'processed_description'] = "".join(chunks_text)

print(industry_data['processed_description'][15])

# apply clean_text function to the column 'processed_description'
industry_data['processed_description'] = industry_data['processed_description'].apply(lambda x: clean_text(x))

# Tokenize and embed industry descriptions
industry_data['tokens'] = industry_data['processed_description'].apply(lambda x: [token.text for token in nlp(x)])

textile product mills nonapparel textile product sheet towel cut sew textile industry textile product purchase fabric cutting sewing


### Round 1

In [31]:
!pip install rapidfuzz
from rapidfuzz import process, fuzz

In [93]:
import spacy, re

nlp = spacy.load("en_core_web_sm")


def remove_mentions_and_hashtags(text):
    text = re.sub(r"@\w+", "", text)
    text = re.sub(r"#\w+", "", text)
    return text


def remove_numbers(text):
    text = re.sub(r"\d+", "", text)
    return text


def remove_punctuation(text):

    translator = str.maketrans("", "", string.punctuation)

    # Remove punctuation using the translation table
    text_without_punct = text.translate(translator)

    return text_without_punct


def remove_stopwords(text, to_lemmatize: bool = False):
    filtered_sentence = []
    doc = nlp(text)
    for token in doc:
        if token.is_stop == False:
            if to_lemmatize:
                filtered_sentence.append(token.lemma_)
            else:
                filtered_sentence.append(token.text)

    return " ".join(filtered_sentence)


def clean_text(text, to_lemmatize: bool = True):

    # Standardize text
    # text = standardize_accented_chars(text)

    # Remove URLs
    # text = re.sub(r"http\S+", "", text)

    # Remove mentions and hashtags
    # text = remove_mentions_and_hashtags(text)

    # Lowercase
    text = text.lower()

    # Remove punctuation
    # text = remove_punctuation(text)

    # Remove numbers
    # text = remove_numbers(text)

    # Remove all the special characters
    text = re.sub(r"\W", " ", text)

    # Remove stopwords
    # text = remove_stopwords(text, to_lemmatize)

    # Substituting multiple spaces with single space
    # text = re.sub(r"\s+", " ", text, flags=re.I)

    # if to_lemmatize:
    #     text = lemmatize(text)

    return text


def lemmatize(text):
    doc = nlp(text)
    text = " ".join([token.lemma_ for token in doc])
    return text

In [20]:
import json

from functools import lru_cache

# Assuming map_company_naics is a dictionary mapping company names to NAICS codes


def read_map(path_json:str):
    f = open(path_json)
    map_company_comercial = json.load(f)
    return map_company_comercial

# map_company = read_map()
map_company_naics = read_map("map_company_naics.json")

map_company_comercial = read_map("map_company_comercial.json")


def read_map(path_json: str):
    f = open(path_json)
    map_company_comercial = json.load(f)
    return map_company_comercial


@lru_cache(maxsize=1024)
def fuzzy_match_naics_round1(company_name, threshold: int = 40):
    matches = process.extractOne(
        company_name, map_company_naics.keys(), scorer=fuzz.WRatio
    )
    if (
        matches and matches[1] > threshold
    ):  # You can adjust the threshold based on your requirements
        return map_company_naics[matches[0]]

    return "abstain"


def map_company_name_to_naics_round1(taxonomy: str):
    return map_company_naics.get(taxonomy, "abstain")

In [116]:
map_company_naics

{'tile store': 'Building Material and Garden Equipment and Supplies Dealers',
 'investment consultant financial advisor': 'Funds, Trusts, and Other Financial Vehicles',
 'damage restoration mold remediation': 'Waste Management and Remediation Services',
 'mortgage broker': 'Credit Intermediation and Related Activities',
 'renewable energy company': 'Utilities',
 'machinery part manufacturer': 'Machinery Manufacturing',
 'car rental': 'Rental and Leasing Services',
 'insurance agent carrier broker': 'Insurance Carriers and Related Activities',
 'metal fabrication service': 'Fabricated Metal Product Manufacturing',
 'garden equipment supply': 'Building Material and Garden Equipment and Supplies Dealers',
 'auto body shop': 'Repair and Maintenance',
 'business consulting': 'Professional, Scientific, and Technical Services',
 'entertainer': 'Performing Arts, Spectator Sports, and Related Industries',
 'pack crate': 'Plastics and Rubber Products Manufacturing',
 'vend machine': 'Food and Be

In [13]:
import tqdm

In [22]:
acc_comercial = 0
acc_naics = 0


for company_name, label in tqdm.tqdm(zip(test_data["business_tags"],test_data["naics_label"])):

    text = company_name.lower()
    text = re.sub(r"\W", " ", text)

    res_naics = predict_industry_label(text, industry_data)[0]

    # res_naics = fuzzy_match_naics_round1(text)
    if res_naics == label:
        acc_naics += 1 


print(f'{acc_naics} from {len(test_data["naics_label"]) }')

0it [00:00, ?it/s]


TypeError: unhashable type: 'list'

In [15]:
test_data = pd.read_csv(
    "/Users/marianluca/Projects/HackingBigNumbers/Tournament/marian/test_data1.csv"
)

In [16]:
test_data

,Unnamed: 0,company_id,commercial_name,answer,business_tags,short_description,description,main_business_category,naics_code,naics_label
0,0,84,Advisory Care,621 Ambulatory Health Care Services,Hygiene Services | One-on-one Attention and Ca...,Advisory Care is a home health care services p...,Advisory Care Pty Ltd is a company that provid...,Home Health Care,621,Ambulatory Health Care Services
1,1,115,PPK Mining Equipment,213 Support Activities for Mining,High & Low Seam Drills | Ventilation Solutions...,PPK Mining Equipment is the solution partner o...,PPK Mining Equipment is a company that special...,Mining & Gas Exploration,213,Support Activities for Mining
2,2,50,First Tracks Lawn Care,561 Administrative and Support Services,Weekly Mowing | Driveway Plowing Services | Mu...,"First Tracks Lawn Care, LLC prides itself on p...","First Tracks Lawn Care, LLC is a lawn care com...",Landscaping & Lawn Services,561,Administrative and Support Services
3,3,92,New Golden Furnishers,337 Furniture and Related Product Manufacturing,Staff Task Computer Chairs Supplier | Mesh Bac...,New Golden Furnishers Co - Manufacturer of Dou...,New Golden Furnishers Co is a furniture manufa...,Office Furniture,337,Furniture and Related Product Manufacturing
4,4,88,Lilac City Septic,562 Waste Management and Remediation Services,Clean-Out Installation | Line Replacement Serv...,Lilac City Septic services residential & comme...,Lilac City Septic Services Co is a family-owne...,Garbage Collection & Waste Disposal,562,Waste Management and Remediation Services
...,...,...,...,...,...,...,...,...,...,...
101,101,28,Purple Pixl,811 Repair and Maintenance,Battery Replacement | Viruses Removal | Front ...,Purple Pixl offers fast & effective repair ser...,Purple Pixl is a technology and electronics re...,Phone Repair & Services,811,Repair and Maintenance
102,102,116,Pasargad Jonub,237 Heavy and Civil Engineering Construction,Infrastructure Project Management | Design and...,Having a brilliant background and well-trained...,Pasargad Jonub is a private joint stock compan...,Civil Engineering Services,237,Heavy and Civil Engineering Construction
103,103,6,Davis Business Machines,"423 Merchant Wholesalers, Durable Goods",Managed IT Services | Document Scanners | Fax ...,"Since May 13, 1963, Davis Business Machines ha...",Davis Business Machines is a company that has ...,Office Supplies,423,"Merchant Wholesalers, Durable Goods"
104,104,24,High Desert Tool,326 Plastics and Rubber Products Manufacturing,Aerospace Molds Supplier | Automotive Molds Di...,High Desert Tool specializes in large mold man...,High Desert Tool is a company that specializes...,Injection Molding Services,326,Plastics and Rubber Products Manufacturing


In [56]:
print(f'{acc_naics} from {len(test_data["naics_label"]) }')

21 from 106


In [22]:
map_company_keyword = read_map(
    "/Users/marianluca/Projects/HackingBigNumbers/Tournament/marian/map_naics_keywords.json"
)

### Round 2

In [77]:
df = pd.read_csv(
    "/Users/marianluca/Projects/HackingBigNumbers/Tournament/marian/cleaned_dataset.csv"
)

In [ ]:
def find_best_matches(keywords_list, df, keyword_column, label_column):
    results = []
    for keyword in keywords_list:
        # Flatten the list of keywords in the DataFrame and find the best match
        all_keywords = sum(df[keyword_column].tolist(), [])
        best_match = process.extractOne(keyword, all_keywords, score_cutoff=60)

        if best_match:
            # Find the label corresponding to the best match
            match_keyword = best_match[0]
            label = df[df[keyword_column].apply(lambda x: match_keyword in x)][
                label_column
            ].iloc[0]
            results.append((keyword, match_keyword, label))
        else:
            results.append((keyword, None, "No Match Found"))

    return results


# Apply the function
matching_results = find_best_matches(keywords_list, df, "keywords", "labels")
print(matching_results)

### Round 5

In [18]:

f = open("map_business_to_naics.json")
label_dict = json.load(f)

def map_bussines_to_naics(taxonomy: str):
    return label_dict.get(taxonomy, "abstain")



### Test something

## API calls

In [18]:
import time
import requests
import json
import pandas as pd

base_url = 'http://116.202.111.229:8000'
api_key = 'VPLrVk4hSZMdGrW2wAP0GTpsV2Jsdx5Z'

headers = {
    'x-api-key': api_key
}

rounds = 5
current_round = 1

filename = 'extra_data.csv'
data_columns = ['commercial_name', 'business_tags', 'short_descripti', 'description', 'main_business_category', 'naics']

index = 0

while True:
    # Get a new hint for current company or get the first hint for a new company after calling /evaluate/reset
    while current_round != 6:
        response = requests.get(f"{base_url}/evaluate/hint", headers=headers)

        print(response.status_code, response.json())

        # Get the hint
        obj_response = response.json()['hint']
        hint = obj_response["hint"]
        level = obj_response["level"]

        if level == 1:
            print("do something")

        elif level == 2:
            print("do something")

        elif level == 3:
            print("do something")

        elif level == 4:
            print("do something")

        elif level == 5:
            answer1 = map_bussines_to_naics(
                hint
            )

        # predict based off given hint
        time.sleep(1)

        final_answer = "abstain"

        if current_round == 2: #keywords
            input_keywords = hint
            # final_answer = "".join(predict_industry_label(input_keywords, industry_data)[0].split(" ")[1:])
            final_answer = predict_industry_label(input_keywords, industry_data)[0]

        # Post your answer for current hint
        data = {
            'answer': final_answer
        }

        print("Guess: ", final_answer)

        response = requests.post(f"{base_url}/evaluate/answer", json=data, headers=headers)

        print(response.status_code, response.json())

        print("")

        current_round = current_round + 1
        time.sleep(1)

    print("----------------------------------------------------------------------------\n")

    # Get hints about a new company
    current_round = 1
    index = index + 1

    response = requests.get(f"{base_url}/evaluate/reset", headers=headers)

    print(response.status_code, response.json())

    # get response from repsonse
    response = response.json()['response']

    if response == 'OK':
        time.sleep(1)
        continue
    else:
        # get float from string
        time_to_wait = float(response.split(" ")[-2])

        # round to the upper integer
        time_to_wait = int(time_to_wait) + 1

        time.sleep(time_to_wait)

        response = requests.get(f"{base_url}/evaluate/reset", headers=headers)
        time.sleep(1)

200 {'company_id': 147, 'level': 1, 'hint': 'Branscomb & Associates'}


Guess:  abstain
200 {'result': 'wrong', 'score': -50.0, 'answer': '524 Insurance Carriers and Related Activities'}

200 {'company_id': 147, 'level': 2, 'hint': 'Trailer Insurance | Boat Insurance | Insurance and Financial Solutions | Roadside Assistance Services | Motorcycle Insurance | Home Insurance | Financial Services Provider | Merchant Policies | Financial Services | Off Road Vehicle Insurance | Motorhome Insurance'}


NameError: name 'predicted_label' is not defined